In [163]:
import os
from pathlib import Path
from deepface import DeepFace
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

In [171]:
from IPython.utils import io

In [50]:
# try:
#     os.mkdir("dataset")
#     os.mkdir("dataset/train")
#     os.mkdir("dataset/test")
# except FileExistsError:
#     pass

In [54]:
# TEST_SIZE = 0.333
# persons = []
# for dir_ in os.listdir('lfw/'):
#     folder = Path('lfw/' + dir_)
#     num_photos = len(list(folder.iterdir()))
    
#     # Если изображений больше, чем 2 отберем их для train/test выборки
#     if num_photos > 2:
#         z = 0
        
#         # Сколько изображений взять для теста
#         n_tests_images = int(num_photos * TEST_SIZE)
#         n_tests_images = 1 if n_tests_images==0 else n_tests_images
        
#         persons.append(dir_)
        
#         for img in os.listdir('lfw/' + dir_):
#             src = Path(f'lfw/{dir_}/{img}')
            
#             # Распределение файлов в зависимсоти от размера test части
#             if z < n_tests_images:
#                 dest_to_test = Path(f'dataset/test/{img}')
#                 dest_to_test.write_bytes(src.read_bytes())
#                 z += 1
#             else:
#                 dest_to_train = Path(f'dataset/train/{img}')
#                 dest_to_train.write_bytes(src.read_bytes())

In [175]:
# Создание трейнового массива

X_train = np.empty((1, 2622))
y_train = np.empty(1)
for filename in tqdm(os.listdir('dataset/train')):
    path = f'dataset/train/{filename}'
    
    try:
        # Перехват вывода исполнения команды в контексте
        with io.capture_output() as captured:
            embedding = np.array(DeepFace.represent(img_path = path)).reshape(1, -1)
    # Если не удалось распознать лицо, то запишем NaN
    except ValueError:
        nan_arr = np.empty((1,2622))
        nan_arr[:] = np.nan
        X_train = np.append(X_train, nan_arr, axis=0)
        y_train = np.append(y_train, filename)
        continue
    
    X_train = np.append(X_train, embedding, axis=0)
    y_train = np.append(y_train, filename)

100%|██████████| 5427/5427 [27:56<00:00,  3.24it/s]


In [176]:
X_train.nbytes

113857728

In [177]:
with open('X_train.npy', 'wb') as f:
    np.save(f, X_train)
with open('y_train.npy', 'wb') as f:
    np.save(f, y_train)